In [1]:
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
import torch

import bvcopula
import utils

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
path_models = '/home/nina/models'
path_exp = '/home/nina/VRData/Processing/pkls'
beh = 5

In [5]:
animal = 'ST260'
N_max = 104
dayN = 2
day_name = f"Day{dayN}"
exp_pref = f"{animal}_{day_name}"
device = torch.device('cuda:0')

In [6]:
X,_ = utils.load_experimental_data(path_exp, animal, day_name, 0, 1)
S = torch.tensor(X).float().squeeze().to(device)

In [7]:
# points = torch.linspace(0,1,20)
# likelihoods = utils.get_likelihoods(f"{path_models}/{exp_pref}/summary.pkl",n1,n2)
# weights_file = f"{path_models}/{exp_pref}/model_{n1}-{n2}.pth"
# %time model = utils.get_model(weights_file, likelihoods, torch.device('cpu')) 
# %time (ym, y, dy) = model.MI(points)
# plt.plot(np.linspace(0,160,len(y)),y)
# plt.fill_between(np.linspace(0,160,len(y)),y-dy,y+dy, alpha=0.3)
# plt.plot(np.linspace(0,160,len(ym)),ym)
# plt.ylim(-.5,0)

In [ ]:
import time

f_mc_size = 5

def get_MI(n1,n2):
    likelihoods = utils.get_likelihoods(f"{path_models}/{exp_pref}/summary.pkl",n1,n2)
    if (len(likelihoods)==1) & (likelihoods[0].name=='Independence'):
        return (0,0)
    else:
        weights_file = f"{path_models}/{exp_pref}/model_{n1}-{n2}.pth"

        model = utils.get_model(weights_file, likelihoods, device) 
        with torch.no_grad():
            Fs = model(S).rsample(torch.Size([f_mc_size])) 
            #[samples_f, copulas, stimuli(positions)]
            MI,sem=model.likelihood.stimMI(S,Fs,s_mc_size=200,r_mc_size=20,sR_mc_size=3000,sem_tol=5*1e-3)
            #print(f"{MI.mean().item():.3}±{MI.std().item():.3} -- {(sem.max()/(MI.max()-MI.min())).item():.3}")

        return (MI.mean(),MI.std())

MI = np.zeros((N_max+beh,N_max+beh,2))

start_time = time.time()
for n1 in range(-5,N_max-1):
    t1 = time.time()
    k=0
    for n2 in range(n1+1,N_max):
        MI[n1+beh,n2+beh,:] = get_MI(n1,n2)
        if MI[n1+beh,n2+beh,0]!=0:
            k+=1
    if k!=0:
        print(n1,' ',(time.time()-t1)/k,' s')
    
total_time = (time.time() - start_time)/60
print(f"Took {total_time:.1f} min overall")

In [ ]:
with open(f"{path_models}/MI_measures/{exp_pref}_copulaMI.pkl",'wb') as f:
    pkl.dump(MI,f)